In [21]:
#importing the required modules.
import random
import json
import pickle
import numpy as np
import nltk
  
from keras.models import Sequential
from nltk.stem import WordNetLemmatizer
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
  
  
lemmatizer = WordNetLemmatizer()
  
# reading the json.intense file
intents = json.loads(open("intents.json").read())
  
# creating empty lists to store data
words = []
classes = []
documents = []
ignore_letters = ["?", "!", ".", ","]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # separating words from patterns
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)  # and adding them to words list
          
        # associating patterns with respective tags
        documents.append(((word_list), intent['tag']))
  
        # appending the tags to the class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
  
# storing the root words or lemma
words = [lemmatizer.lemmatize(word)
         for word in words if word not in ignore_letters]
words = sorted(set(words))
  
# saving the words and classes list to binary files
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [22]:
# we need numerical values of the
# words because a neural network
# needs numerical values to work with
training = []
output_empty = [0]*len(classes)
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(
        word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
          
    # making a copy of the output_empty
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)
  
# splitting the data
train_x = list(training[:, 0])
train_y = list(training[:, 1])

/tmp/ipykernel_2874/330460699.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [23]:
# creating a Sequential machine learning model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ),
                activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), 
                activation='softmax'))
  
# compiling the model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y),
                 epochs=200, batch_size=5, verbose=1)
  
# saving the model
model.save("chatbotmodel.h5", hist)
  
# print statement to show the 
# successful training of the Chatbot model
print("Yay!")

Epoch 1/200
64/64 [==============================] - 0s 1ms/step - loss: 4.3698 - accuracy: 0.0219
Epoch 2/200
64/64 [==============================] - 0s 1ms/step - loss: 4.2044 - accuracy: 0.0594
Epoch 3/200
64/64 [==============================] - 0s 1ms/step - loss: 3.9806 - accuracy: 0.0719
Epoch 4/200
64/64 [==============================] - 0s 1ms/step - loss: 3.8100 - accuracy: 0.1156
Epoch 5/200
64/64 [==============================] - 0s 1ms/step - loss: 3.6310 - accuracy: 0.1437
Epoch 6/200
64/64 [==============================] - 0s 1ms/step - loss: 3.4288 - accuracy: 0.1937
Epoch 7/200
64/64 [==============================] - 0s 1ms/step - loss: 3.2765 - accuracy: 0.2313
Epoch 8/200
64/64 [==============================] - 0s 1ms/step - loss: 3.0469 - accuracy: 0.2844
Epoch 9/200
64/64 [==============================] - 0s 1ms/step - loss: 2.8353 - accuracy: 0.3125
Epoch 10/200
64/64 [==============================] - 0s 1ms/step - loss: 2.6751 - accuracy: 0.3219
Epoch 11/